### Инструкция по выполнению

    1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут.
    2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.
    3. Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).
    4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.
    5. Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

Ответ на каждое задание — текстовый файл, содержащий ответ в первой строчке. Обратите внимание, что отправляемые файлы не должны содержать перевод строки в конце. Данный нюанс является ограничением платформы Coursera. Мы работаем над тем, чтобы убрать это ограничение.

In [14]:
from sklearn import datasets

newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(newsgroups.data)

In [27]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


grid = {'C': np.power(10.0, np.arange(-5, 6))}
kf = KFold(n_splits=5, shuffle=True, random_state=42)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=kf)
gs.fit(X, newsgroups.target)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVC(kernel='linear', random_state=241),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             scoring='accuracy')

In [31]:
#for a in gs.grid_scores_:
#    print(a.mean_validation_score, a.parameters)

gs.best_params_, gs.best_score_

({'C': 1.0}, 0.9915997683989797)

In [56]:
import pandas as pd

tf_idf = pd.DataFrame(
    np.transpose(abs(gs.best_estimator_.coef_.toarray())),
    index=np.asarray(vectorizer.get_feature_names()),
    columns=['col1']
)
tf_idf.sort_values(by='col1', ascending=False, inplace=True)

In [62]:
top_ten = tf_idf.index[:10]
top_ten = top_ten.sort_values()

answer=''
for i in top_ten:
    answer += i + ' '
answer

'atheism atheists bible god keith moon religion sci sky space '

In [63]:
file = open('/home/topcoder2k/HSE_ML/HSE_ML_week3_answers/top_ten_words.txt', 'w')
file.write('atheism atheists bible god keith moon religion sci sky space')
file.close()